In [13]:
import numpy as np
import open3d as o3d

In [19]:
pc_ply = o3d.io.read_point_cloud("data\BunnyMesh.ply")
print('PLY file loaded')
# Get the point cloud data as a NumPy array
point_cloud_np = np.asarray(pc_ply.points)
# Extract X, Y, and Z coordinates
x = point_cloud_np[:, 0]
y = point_cloud_np[:, 1]
z = point_cloud_np[:, 2]
# Print or use the X, Y, and Z coordinates as needed
print('X coordinates:', x)
print('Y coordinates:', y)
print('Z coordinates:', z)
# Visualization (optional)
o3d.visualization.draw_geometries([pc_ply])


PLY file loaded
X coordinates: [-0.0378297 -0.0447794 -0.0680095 ... -0.0704544 -0.0310262 -0.0400442]
Y coordinates: [0.12794  0.128887 0.151244 ... 0.150585 0.153728 0.15362 ]
Z coordinates: [ 0.00447467  0.00190497  0.0371953  ... -0.0434585  -0.00354608
 -0.00816685]


### Algorithm

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  
from sklearn.neighbors import NearestNeighbors

def point_cloud_3d(x, y, z):
    num_points = len(x)  # Assuming x, y, and z have the same length
    k_value = 4  # Number of nearest neighbors for k-NN

    # Create an array of point coordinates
    points = np.column_stack((x, y, z))

    # Create a NearestNeighbors model with k=k_value
    knn_model = NearestNeighbors(n_neighbors=k_value + 1)  # +1 to include the point itself
    knn_model.fit(points)

    # Find indices of points with k-NN distances greater than the average
    distances, indices = knn_model.kneighbors(points)
    average_distance = np.mean(distances[:, 1:])  # Exclude the distance to itself

    # Indices of points with distances greater than the average
    indices_above_avg = np.where(distances[:, 1:] > average_distance)

    # Create a 3D scatter plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot all points in blue
    ax.scatter(x, y, z, color='blue', label='All Points')

    # Mark points with distances greater than average as red
    ax.scatter(x[indices_above_avg[0]], y[indices_above_avg[0]], z[indices_above_avg[0]], color='red', s=10, label='Dist > Average')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.legend()
    plt.show()



In [4]:
from sklearn.neighbors import NearestNeighbors

def point_cloud_3d(x, y, z):
    num_points = len(x)  # Assuming x, y, and z have the same length
    k_value = 1 # Number of nearest neighbors for k-NN

    # Create an array of point coordinates
    points = np.column_stack((x, y, z))

    # Create a NearestNeighbors model with k=k_value
    knn_model = NearestNeighbors(n_neighbors=k_value + 1)  # +1 to include the point itself
    knn_model.fit(points)

    # Find indices of points with k-NN distances greater than the average
    distances, indices = knn_model.kneighbors(points)
    average_distance = np.mean(distances[:, 1:])  # Exclude the distance to itself

    # Create an Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np.column_stack((x, y, z)))

    # Assign colors based on distances
    colors = np.zeros((num_points, 3))  # Initialize colors as blue
    above_avg_indices = indices_above_avg(distances, average_distance)
    colors[above_avg_indices] = [1, 0, 0]  # Set color to red for points above average distance

    pcd.colors = o3d.utility.Vector3dVector(colors)

    # Create Open3D visualization window
    o3d.visualization.draw_geometries([pcd])

    print("Average Distance between Data Points:", average_distance)

def indices_above_avg(distances, average_distance):
    return np.where(distances[:, 1:] > average_distance)[0]

point_cloud_3d(x, y, z)


[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
Average Distance between Data Points: 0.004740746752983856


### these are finding normals


In [17]:
import open3d as o3d
import numpy as np

point_cloud = o3d.io.read_point_cloud('data\BunnyMesh.ply')

k_neighbors = 1

# Estimate normals using KNN algorithm
point_cloud.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamKNN(k_neighbors)
)

# Access the estimated normals
normals = np.asarray(point_cloud.normals)

print(normals)
print(len(normals))


[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
35947
